<a href="https://colab.research.google.com/github/hariprasad7/hari_INFO5731_Spring2021/blob/main/In_Class_exercises_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In class exercise 10 (20 points in total, 4/16/2021)

The purpose of the exercise is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 
    

In [1]:
import pandas as pd
import gensim
df=pd.read_csv("/content/Amazon_Unlocked_Mobile.csv")
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import Word
nltk.download('wordnet')
from nltk.stem import PorterStemmer
st = PorterStemmer()
df = df[df['Reviews'].notnull()]
stop = stopwords.words('english')
df['after_punct_less'] = df['Reviews'].str.replace('[^\w\s].#','')
df['after_rm_stopwords'] =df['after_punct_less'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['after_numerics']=df['after_rm_stopwords'].str.replace('[0-9]','')
df['after_lowercasing'] =df['after_numerics'].apply(lambda x: " ".join(x.lower() for x in x.split()))

df['after_stemming']=df['after_lowercasing'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['cleaned_text'] = df['after_stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['cleaned_text']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


0         i feel lucki found use (phone u & use hard all...
1         nice phone, nice grade pantach revue. veri cle...
2                                                 veri plea
3           it work good goe slow sometim good phone i love
4         great phone replac lost phone. the thing volum...
                                ...                        
413835                         anoth great deal great price
413836                                                   ok
413837             pas everi drop test onto porcelain tile!
413838          i return meet need seem good select others.
413839    onli downsid appar verizon longer use v-cast m...
Name: cleaned_text, Length: 413778, dtype: object

In [2]:
df_1=(df.sample(n=4000)).reset_index()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(df_1['cleaned_text'].values)
tfidf.shape

from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters = 5, n_jobs = -1,random_state=99)
model_tf.fit(tfidf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=99, tol=0.0001, verbose=0)

In [3]:
labels_tf = model_tf.labels_
cluster_center_tf=model_tf.cluster_centers_
terms = tfidf_vect.get_feature_names()
terms[1:5]
df_2 = df_1
df_2['Tfidf Clus Label'] = model_tf.labels_
df_2.head(5)

,index,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,after_punct_less,after_rm_stopwords,after_numerics,after_lowercasing,after_stemming,cleaned_text,Tfidf Clus Label
0,58331,Apple iPhone 6 Plus Factory Unlocked Cellphone...,Apple,605.00,1,"16gb is not enough, you find out after about 1...",3.0,"16gb is not enough, you find out after about 1...","16gb enough, find 1 month, cant return exchang...","gb enough, find month, cant return exchange i...","gb enough, find month, cant return exchange it...","gb enough, find month, cant return exchang it!...","gb enough, find month, cant return exchang it!...",1
1,137650,"BLU Studio 5.0 C HD Unlocked Cellphone, Black",BLU,173.44,2,"This phone has nice features, but does not hav...",0.0,"This phone has nice features, but does not hav...","This phone nice features, storage space claims...","This phone nice features, storage space claims...","this phone nice features, storage space claims...","thi phone nice features, storag space claims. ...","thi phone nice features, storag space claims. ...",1
2,400484,Studio 5.0 E Quad Band Unlocked (White) (Disco...,BLU,104.99,5,"excellent article, the best price and delivery...",1.0,"excellent article, the best price and delivery...","excellent article, best price delivery time fa...","excellent article, best price delivery time fa...","excellent article, best price delivery time fa...","excel article, best price deliveri time fast r...","excel article, best price deliveri time fast r...",1
3,275060,Nokia Lumia 920 32GB Unlocked GSM Windows 8 Sm...,NaN,99.99,5,Excellent cellphone! I didn't find any issues ...,0.0,Excellent cellphone! I didn't find any issues ...,Excellent cellphone! I find issues far. I thou...,Excellent cellphone! I find issues far. I thou...,excellent cellphone! i find issues far. i thou...,excel cellphone! i find issu far. i thought wo...,excel cellphone! i find issu far. i thought wo...,1
4,345122,Samsung Galaxy S4 Mini I257 16GB Unlocked GSM ...,Samsung,84.95,4,Really great phone. I couldn't believe it work...,3.0,Really great phone. I couldn't believe it work...,Really great phone. I believe worked t-mobile ...,Really great phone. I believe worked t-mobile ...,really great phone. i believe worked t-mobile ...,realli great phone. i believ work t-mobil sim ...,realli great phone. i believ work t-mobil sim ...,1


In [4]:
df_2.groupby(['Tfidf Clus Label'])['cleaned_text'].count()

Tfidf Clus Label
0     206
1    3213
2     160
3     250
4     171
Name: cleaned_text, dtype: int64

In [6]:
print("Top terms per cluster:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(1,5):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :5]:
        print(' %s' % terms[ind], end='')
        print()

Top terms per cluster:
Cluster 1: phone
 it
 work
 the
 use
Cluster 2: love
 it
 phone
 great
 thank
Cluster 3: great
 phone
 work
 product
 price
Cluster 4: good
 veri
 product
 phone
 it


In [7]:
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import gensim
i=0
list_of_sent=[]
for sent in df_1['cleaned_text'].values:
    list_of_sent.append(sent.split())
w2v_model=gensim.models.Word2Vec(list_of_sent,size=100, workers=4)
import numpy as np
sent_vectors = [];
for sent in list_of_sent:
    sent_vec = np.zeros(100)
    cnt_words =0;
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


(4000, 100)

In [16]:
minPts = 2 * 100
model = DBSCAN(eps = 8, min_samples = minPts, n_jobs=-2)
model.fit(sent_vectors)
df_1['AVG-W2V Clus Label'] = model.labels_
df_1.head(2)

,index,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,after_punct_less,after_rm_stopwords,after_numerics,after_lowercasing,after_stemming,cleaned_text,Tfidf Clus Label,AVG-W2V Clus Label
0,58331,Apple iPhone 6 Plus Factory Unlocked Cellphone...,Apple,605.00,1,"16gb is not enough, you find out after about 1...",3.0,"16gb is not enough, you find out after about 1...","16gb enough, find 1 month, cant return exchang...","gb enough, find month, cant return exchange i...","gb enough, find month, cant return exchange it...","gb enough, find month, cant return exchang it!...","gb enough, find month, cant return exchang it!...",1,0
1,137650,"BLU Studio 5.0 C HD Unlocked Cellphone, Black",BLU,173.44,2,"This phone has nice features, but does not hav...",0.0,"This phone has nice features, but does not hav...","This phone nice features, storage space claims...","This phone nice features, storage space claims...","this phone nice features, storage space claims...","thi phone nice features, storag space claims. ...","thi phone nice features, storag space claims. ...",1,0


In [17]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')  
Agg=cluster.fit_predict(sent_vectors)
df_1['AVG-W2V Clus Label'] = cluster.labels_
df_1.head(2)
df_1.groupby(['AVG-W2V Clus Label'])['cleaned_text'].count()
for i in range(5):
    print("Example of 2 reviews assigned to cluster ", i)
    print("-" * 70)
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][0]]['cleaned_text'])
    print('\n')
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][1]]['cleaned_text'])
    print('\n')
    print("_" * 70)

Example of 2 reviews assigned to cluster  0
----------------------------------------------------------------------
gb enough, find month, cant return exchang it!!! mad


wonder


______________________________________________________________________
Example of 2 reviews assigned to cluster  1
----------------------------------------------------------------------
excellen


fate


______________________________________________________________________
Example of 2 reviews assigned to cluster  2
----------------------------------------------------------------------
excel cellphone! i find issu far. i thought would come headphon .


realli great phone. i believ work t-mobil sim card came android . pre-installed. the fact gb free memori also made best phone i ever had! take one star away screen got slightli scratch tri clean it. i never phone sensit screen, i immedi bought pack ot screen protectors. otherwise, great phone!


__________________________________________________________________

In one paragraph, please compare K means, DBSCAN and Hierarchical clustering. 

In [13]:
'''
The k-means Clustering is a vector quantization method originating in signal processing that aims to partition of n observations into k clusters, with each observation belonging to the cluster with the nearest mean, which serves as the cluster's prototype. The results show that k-means are easier to comprehend than DBSCAN. Also missing values in the data set can be regulated by the DBSCAN. As Hierarchial clustering have lot of noise, we can say k-means are best and optimal.
'''

"\nThe k-means Clustering is a vector quantization method originating in signal processing that aims to partition of n observations into k clusters, with each observation belonging to the cluster with the nearest mean, which serves as the cluster's prototype. The results show that k-means are easier to comprehend than DBSCAN. Also missing values in the data set can be regulated by the DBSCAN. As Hierarchial clustering have lot of noise, we can say k-means are best and optimal.\n"